# Generalised Bernoulli map with various number formats

Revisiting BM Boghosian, PV Coveney, and H Wang. *A new pathology in the Simulation of Chaotic Dynamical Systems on Digital Computers*, **Adv. Theory Simul.** 2019, 2, 1900125, DOI: 10.1002/adts.201900125

### 0. Load packages

In [1]:
using PyPlot, FileIO, JLD2, Printf
using StochasticRounding, SoftPosit, LogFixPoint16s
using StatsBase, LinearAlgebra, Statistics
LogFixPoint16s.set_nfrac(10)   # use the 16-bit logfix format similar to Float16

┌ Warning: LogFixPoint16 was changed to 5 integer and 10 fraction bits.
└ @ LogFixPoint16s /Users/milan/.julia/packages/LogFixPoint16s/TGYbV/src/change_format.jl:25


### 1. Functions
Define the generalised Bernoulli map

$\quad x_{j+1} = f_\beta(x_j) = \beta x_j \mod 1$

with $\beta > 1$.

In [2]:
function bernoulli_orbit( x::T,            # initial condition
                          β::T,            # Bernoulli parameter
                          N0::Int,         # spin up iterations
                          N::Int) where T  # max period length
    oone = one(T)                  # 1 in format T
    
    # SPIN UP
    for i in 1:N0
        x = β*x
        x = x >= oone ? x-oone : x # x mod 1
    end
    
    # CHECK FOR PERIOD LENGTH
    x0 = x                         # new initial condition
    n = 0                          # orbit length (0 = not found yet)
    j = 0                          # iteration counter
    
    while n == 0 && j < N
        j += 1
        x = β*x
        x = x >= oone ? x-oone : x # x mod 1
        n = x0 == x ? j : 0        # check for periodicity
    end

    return n,x                     # return orbit length and last x that's on the orbit
end

bernoulli_orbit (generic function with 1 method)

In [3]:
function bernoulli_map(x::T,β::T,N::Int) where T
    oone = one(T)                  # 1 in format T
    xout = Array{T,1}(undef,N+1)   # preallocate
    xout[1] = x                    # store initial condition
    for i in 2:N+1
        x = β*x
        while x >= oone            # x mod 1
            x = x - oone
        end
        xout[i] = x                # store iteration
    end
    return xout
end

bernoulli_map (generic function with 1 method)

In [4]:
function find_orbits(   ::Type{T},                  # Number format
                        β::Real;                    # Bernoulli parameter
                        n::Int=100000,              # n initial conditions
                        N0::Int=50000,              # spinup
                        Nmax::Int=1000000) where T  # maximum orbit length

    println("β = $β, $T")
    
    O = fill(0,n)           # array of orbit period lengths for each IC
    X = Array{T}(undef,n)   # for each orbit one x on the orbit

    for i in 1:n            # for n ICs calculate orbit lengths & x
        O[i],X[i] = bernoulli_orbit(T(rand()),T(β),N0,Nmax)
    end

    # find unique orbit lengths via Set
    orbit_lengths = sort([o for o in Set(O)])

    iol = 1
    for (iol,ol) in enumerate(orbit_lengths)

        # all x for orbit length ol, use as initial condition
        Xini = X[O .== ol]     
        S = Set{T}()         # preallocate a set to check orbits of their uniqueness

        for x in Xini        # per IC calculate all points on that orbit
            xorbit = bernoulli_map(x,T(β),ol)
            S = S ∪ Set(minimum(xorbit))   # and identify by their min x
        end
    
        p = length(Xini)/n        # size of the basin of attraction
        x0s = [s for s in S]      # array of min x to identify unique orbits
                                  # if >1 orbit exists with same length length(x0s)>1
        
        # print orbit informtation
        s1 = "Orbit $(@sprintf("%2d",iol)): length = $(@sprintf("%6d",ol)), "
        s2 = "x₀ = $(@sprintf("%16s",repr(x0s[1]))), "
        s3 = "p = $(@sprintf("%.6f",p))"
        println(s1*s2*s3)
        iol += 1
        
        # in case there are several orbits of same period length, print their info
        if length(x0s) > 1
            for x0 in x0[2:end]
                s2 = "x₀ = $(@sprintf("%.14f",x0)), "
                println(s1*s2)
                iol += 1
            end
        end
    end
end

find_orbits (generic function with 1 method)

In [6]:
find_orbits(Float32,4//3,n=1000000)

β = 4//3, Float32
Orbit  1: length =      1, x₀ =            0.0f0, p = 0.002991
Orbit  2: length =      6, x₀ =     0.21651316f0, p = 0.000006
Orbit  3: length =     48, x₀ =     0.06936097f0, p = 0.000108
Orbit  4: length =     49, x₀ =    0.048506975f0, p = 0.000267
Orbit  5: length =    153, x₀ =      0.0189718f0, p = 0.000358
Orbit  6: length =    336, x₀ =   0.0012991428f0, p = 0.019723
Orbit  7: length =   3304, x₀ =  0.00014281273f0, p = 0.463223
Orbit  8: length =   4790, x₀ =   0.0004069805f0, p = 0.513324


In [7]:
find_orbits(Float32,8//7,n=1000000)

β = 8//7, Float32
Orbit  1: length =     16, x₀ =     0.13387299f0, p = 0.000031
Orbit  2: length =     26, x₀ =    0.032056212f0, p = 0.000004
Orbit  3: length =     51, x₀ =      0.1009475f0, p = 0.000010
Orbit  4: length =     93, x₀ =    0.039530635f0, p = 0.000174
Orbit  5: length =    102, x₀ =    0.009067774f0, p = 0.000090
Orbit  6: length =    339, x₀ =    0.016980052f0, p = 0.000999
Orbit  7: length =   2082, x₀ =   0.0020537376f0, p = 0.038340
Orbit  8: length =   2676, x₀ =   0.0014624596f0, p = 0.071857
Orbit  9: length =   3837, x₀ =  0.00040769577f0, p = 0.070869
Orbit 10: length =  10889, x₀ =  0.00067329407f0, p = 0.817626


In [8]:
find_orbits(Float32,3//2,n=1000000)

β = 3//2, Float32
Orbit  1: length =      5, x₀ =     0.15165877f0, p = 0.000002
Orbit  2: length =     19, x₀ =     0.06243241f0, p = 0.009374
Orbit  3: length =     39, x₀ =    0.010771394f0, p = 0.000011
Orbit  4: length =     69, x₀ =   0.0052359104f0, p = 0.000376
Orbit  5: length =     86, x₀ =    0.037434578f0, p = 0.000730
Orbit  6: length =    285, x₀ =    0.008414507f0, p = 0.000743
Orbit  7: length =    297, x₀ =     5.2928925f-5, p = 0.008579
Orbit  8: length =    313, x₀ =      6.747246f-5, p = 0.006114
Orbit  9: length =    445, x₀ =   0.0024217367f0, p = 0.001186
Orbit 10: length =    494, x₀ =   0.0057212114f0, p = 0.046928
Orbit 11: length =    517, x₀ =   0.0037292242f0, p = 0.045287
Orbit 12: length =    982, x₀ =    0.002391696f0, p = 0.061751
Orbit 13: length =   5228, x₀ =     4.1007996f-5, p = 0.818919


In [9]:
find_orbits(Float32,9//8,n=1000000)

β = 9//8, Float32
Orbit  1: length =      1, x₀ =            0.0f0, p = 0.000001
Orbit  2: length =     21, x₀ =     0.09205365f0, p = 0.000002
Orbit  3: length =     22, x₀ =     0.08099699f0, p = 0.000025
Orbit  4: length =     78, x₀ =    0.003218174f0, p = 0.000011
Orbit  5: length =   1013, x₀ =   0.0010734797f0, p = 0.030015
Orbit  6: length =   5258, x₀ =  0.00027680397f0, p = 0.969946


In [5]:
find_orbits(Float32,7//6,n=1000000)

β = 7//6, Float32
Orbit  1: length =      1, x₀ =            0.0f0, p = 0.000001
Orbit  2: length =     13, x₀ =     0.15580297f0, p = 0.000007
Orbit  3: length =     87, x₀ =    0.027895927f0, p = 0.000033
Orbit  4: length =   1486, x₀ =   0.0011222363f0, p = 0.003857
Orbit  5: length =   3185, x₀ =   0.0018539429f0, p = 0.041454
Orbit  6: length =  21489, x₀ =  0.00025331974f0, p = 0.954648


In [10]:
find_orbits(Float32,5//4,n=1000000)

β = 5//4, Float32
Orbit  1: length =     12, x₀ =     0.07379031f0, p = 0.000013
Orbit  2: length =    137, x₀ =   0.0072830915f0, p = 0.004364
Orbit  3: length =    219, x₀ =    0.005312443f0, p = 0.000601
Orbit  4: length =    228, x₀ =   0.0007648468f0, p = 0.002745
Orbit  5: length =   2117, x₀ =  0.00061380863f0, p = 0.017799
Orbit  6: length =   3763, x₀ =   0.0009752512f0, p = 0.974478


In [11]:
find_orbits(Float32,6//5,n=1000000)

β = 6//5, Float32
Orbit  1: length =      1, x₀ =            0.0f0, p = 0.000001
Orbit  2: length =     16, x₀ =    0.057180643f0, p = 0.000001
Orbit  3: length =    135, x₀ =    0.008523345f0, p = 0.014066
Orbit  4: length =    329, x₀ =   0.0030605793f0, p = 0.004028
Orbit  5: length =    792, x₀ =   0.0016326904f0, p = 0.007740
Orbit  6: length =   2867, x₀ =   0.0010089874f0, p = 0.136498
Orbit  7: length =  10190, x₀ =     5.1259995f-5, p = 0.837666
